In [1]:
import pandas as pd
ratings = pd.read_csv('./ml-latest-small/ratings.csv')    
print(ratings.head(20)) 

userId  movieId  rating  timestamp
0        1        1     4.0  964982703
1        1        3     4.0  964981247
2        1        6     4.0  964982224
3        1       47     5.0  964983815
4        1       50     5.0  964982931
5        1       70     3.0  964982400
6        1      101     5.0  964980868
7        1      110     4.0  964982176
8        1      151     5.0  964984041
9        1      157     5.0  964984100
10       1      163     5.0  964983650
11       1      216     5.0  964981208
12       1      223     3.0  964980985
13       1      231     5.0  964981179
14       1      235     4.0  964980908
15       1      260     5.0  964981680
16       1      296     3.0  964982967
17       1      316     3.0  964982310
18       1      333     5.0  964981179
19       1      349     4.0  964982563


In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv') 
print(movies.head(5))

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [3]:
data = pd.merge(movies, ratings, on='movieId')
print(data.head(5))
data = data[['userId','rating','movieId','title']].sort_values('userId').to_csv('./ml-latest-small/data.csv', index=False, encoding='utf8')
files = pd.read_csv('./ml-latest-small/data.csv')
print(files.head(5))

movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating   timestamp  
0       1     4.0   964982703  
1       5     4.0   847434962  
2       7     4.5  1106635946  
3      15     2.5  1510577970  
4      17     4.5  1305696483  
   userId  rating  movieId                       title
0       1     4.0        1            Toy Story (1995)
1       1     4.0     1777  Wedding Singer, The (1998)
2       1     5.0     1732    Big Lebowski, The (1998)
3       1     3.0     1676    Starship Troopers (1997)
4       1     5.0       50  Usual Suspects, The (1995)


In [4]:
import csv

# read data.csv
content = []
with open('./ml-latest-small/data.csv', encoding='utf8') as fp:  
    cr = csv.reader(fp)
    for row in cr:
        # print(row)
        content.append(row)
print(len(content), content[:2])

# from list to dictionary
data = {}
for line in content[1:]:  # delete the first row
    # print(line)
    if line[0] not in data.keys():
        data[line[0]] = {line[3]:line[1]}
    else:
        data[line[0]][line[3]] = line[1]
print(len(data), data['101'])

100837 [['userId', 'rating', 'movieId', 'title'], ['1', '4.0', '1', 'Toy Story (1995)']]
610 {'Summer of Sam (1999)': '4.0', 'Stir of Echoes (1999)': '3.0', 'Civil Action, A (1998)': '3.0', 'Austin Powers: The Spy Who Shagged Me (1999)': '3.0', 'Rushmore (1998)': '5.0', 'Pushing Tin (1999)': '2.0', 'Gone in 60 Seconds (2000)': '4.0', 'Go (1999)': '4.0', "Boys Don't Cry (1999)": '4.0', 'Ravenous (1999)': '4.0', 'Velvet Goldmine (1998)': '3.0', 'Dead Calm (1989)': '4.0', 'Sixth Sense, The (1999)': '4.0', 'Exotica (1994)': '4.0', 'Mystery Men (1999)': '2.0', 'Man on the Moon (1999)': '5.0', 'Croupier (1998)': '4.0', 'Fight Club (1999)': '5.0', 'Higher Learning (1995)': '2.0', 'Doors, The (1991)': '5.0', 'Deep Blue Sea (1999)': '3.0', 'Bone Collector, The (1999)': '2.0', 'Star Wars: Episode VI - Return of the Jedi (1983)': '4.0', 'Pacific Heights (1990)': '4.0', 'South Park: Bigger, Longer and Uncut (1999)': '4.0', 'Little Voice (1998)': '3.0', 'American Pie (1999)': '4.0', 'Gladiator (200

In [5]:
from math import *

# compute the distance between 2 users
def Euclidean(user1, user2):
    user1_data=data[user1]
    user2_data=data[user2]
    distance = 0
    count = 0
    # find movies that 2 users both give ratings, and compute distance
    for key in user1_data.keys():
        if key in user2_data.keys():
            distance += pow(float(user1_data[key]) - float(user2_data[key]), 2)
            count += 1
    if count > 0:
        distance = sqrt(distance / count)  # [0, 5]
    else:
        distance = 5.5  # if 2 users have a same movie, the greatest distance is 5
 
    return distance  # the smaller, the more similar
 
# compute the similarity with other users
def top10_simliar(userID):
    res = []
    for userid in data.keys():
        if not userid == userID:
            simliarity = Euclidean(userID,userid)
            res.append((userid, simliarity))
    res.sort(key=lambda val:val[1])
    return res[:10]
 
res = top10_simliar('1')
print(res)

[('77', 0.0), ('85', 0.0), ('253', 0.0), ('291', 0.0), ('511', 0.2041241452319315), ('278', 0.28867513459481287), ('550', 0.28867513459481287), ('366', 0.31622776601683794), ('472', 0.3535533905932738), ('49', 0.3779644730092272)]


In [6]:
def recommend(user):
    top_sim_user = top10_simliar(user)[0][0]
    items = data[top_sim_user]
    recommendations = []
    # recommend movies the user did not watch
    for item in items.keys():
        if item not in data[user].keys():
            recommendations.append((item, items[item]))
    recommendations.sort(key=lambda val:val[1], reverse=True)
    return recommendations[:10]  # return 10 movies with highest ratings
 
Recommendations = recommend('1')
print(Recommendations)

[('Incredibles, The (2004)', '5.0'), ('Star Wars: Episode II - Attack of the Clones (2002)', '5.0'), ('Lord of the Rings: The Fellowship of the Ring, The (2001)', '5.0'), ('Harry Potter and the Chamber of Secrets (2002)', '5.0'), ('Spider-Man 2 (2004)', '5.0'), ('Batman Begins (2005)', '5.0'), ('Spider-Man (2002)', '5.0'), ('Lord of the Rings: The Return of the King, The (2003)', '5.0'), ('Dark Knight, The (2008)', '5.0'), ('Lord of the Rings: The Two Towers, The (2002)', '5.0')]
